In [1]:
import time
import pyspark.sql.functions as f

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

from pyspark.sql.functions import array_contains
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import desc



In [2]:
spark = SparkSession.builder.master('"local[*]"').getOrCreate()

spark = SparkSession \
    .builder \
    .appName("Python Spark Assignment") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#### Read data from file and load to the RAM

In [25]:
df_flights = spark.read.csv('/data/lsml/4-5-spark/flights.csv')
df_flights.cache()

df_flights.show(2)
df_flights.count()

+----+------+--------------------+--------------------+---+---+---------+---+----+----+
| _c0|   _c1|                 _c2|                 _c3|_c4|_c5|      _c6|_c7| _c8| _c9|
+----+------+--------------------+--------------------+---+---+---------+---+----+----+
|1185|PG0134|2017-09-10 09:50:...|2017-09-10 14:55:...|DME|BTK|Scheduled|319|null|null|
|3979|PG0052|2017-08-25 14:50:...|2017-08-25 17:35:...|VKO|HMA|Scheduled|CR2|null|null|
+----+------+--------------------+--------------------+---+---+---------+---+----+----+
only showing top 2 rows



33121

In [26]:
df_tickets = spark.read.csv('/data/lsml/4-5-spark/ticket_flights.csv')
df_tickets.cache()

df_tickets.show(2)
df_tickets.count()

+-------------+-----+--------+--------+
|          _c0|  _c1|     _c2|     _c3|
+-------------+-----+--------+--------+
|0005432159776|30625|Business|42100.00|
|0005435212351|30625|Business|42100.00|
+-------------+-----+--------+--------+
only showing top 2 rows



1045726

#### Transform Flights table

##### 1. Calculate delay time in sec

In [27]:
df_flights = df_flights.withColumn('_c2_unix', unix_timestamp(df_flights['_c2'].cast(TimestampType()), 'yyyy-MM-ddThh:mm:ss'))
df_flights = df_flights.withColumn('_c8_unix', unix_timestamp(df_flights['_c8'].cast(TimestampType()), 'yyyy-MM-ddThh:mm:ss'))
df_flights = df_flights.withColumn('_c8-c2_unix', df_flights['_c8_unix'] - df_flights['_c2_unix'])

##### 2. Select only Arrived status

In [28]:
df_flights = df_flights[df_flights['_c6'] == 'Arrived']

##### 3. Drop columns

In [29]:
columns_to_drop = ['_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c2_unix', '_c8_unix']
df_flights = df_flights.drop(*columns_to_drop)

In [30]:
df_flights.show(5)
df_flights.count()

+---+------+-----------+
|_c0|   _c1|_c8-c2_unix|
+---+------+-----------+
|  1|PG0405|        540|
|  2|PG0404|         60|
|  3|PG0405|        240|
| 14|PG0402|        180|
| 15|PG0402|        360|
+---+------+-----------+
only showing top 5 rows



16707

##### 4. Select Flight's with Flight Number, which is in Tickets

In [31]:
df_tickets_grouped = df_tickets.groupby('_c1').count()
df_tickets_grouped = df_tickets_grouped.withColumnRenamed('_c1', '_c1_tmp')
df_tickets_grouped.cache()

DataFrame[_c1_tmp: string, count: bigint]

In [32]:
df_flights = df_flights.join(df_tickets_grouped, df_flights['_c0'] == df_tickets_grouped['_c1_tmp'])
columns_to_drop = ['_c1_tmp', 'count']
df_flights = df_flights.drop(*columns_to_drop)
df_flights.show(5)

+---+------+-----------+
|_c0|   _c1|_c8-c2_unix|
+---+------+-----------+
|  1|PG0405|        540|
|  2|PG0404|         60|
|  3|PG0405|        240|
| 17|PG0404|          0|
| 18|PG0405|        240|
+---+------+-----------+
only showing top 5 rows



In [33]:
df_flights.count()

11438

#### Calculate AVG DELAY OF DEPARTURE TIME

In [34]:
df_avg = df_flights.groupby('_c1').avg('_c8-c2_unix')
df_avg.show(5)
df_avg.count()

+------+------------------+
|   _c1|  avg(_c8-c2_unix)|
+------+------------------+
|PG0144|193.84615384615384|
|PG0325|             160.0|
|PG0503| 574.8387096774194|
|PG0278|1832.9032258064517|
|PG0088|             280.0|
+------+------------------+
only showing top 5 rows



470

In [36]:
df_avg[df_avg['avg(_c8-c2_unix)'] == 0].show()

+------+----------------+
|   _c1|avg(_c8-c2_unix)|
+------+----------------+
|PG0587|             0.0|
+------+----------------+



### Get tickets from Ticket table where flight status is Arrived

In [13]:
df_tickets_arrived = df_tickets.join(df_flights, df_tickets['_c1'] == df_flights['_c0']).drop(df_flights['_c0']).drop(df_flights['_c8-c2_unix'])

df_tickets_arrived.show(5)
df_tickets_arrived.count()

+-------------+-----+--------+--------+------+
|          _c0|  _c1|     _c2|     _c3|   _c1|
+-------------+-----+--------+--------+------+
|0005434877632|10096|Business|49700.00|PG0186|
|0005434878408|10096|Business|49700.00|PG0186|
|0005434878496|10096|Business|49700.00|PG0186|
|0005434878162|10096|Business|49700.00|PG0186|
|0005434877976|10096|Business|49700.00|PG0186|
+-------------+-----+--------+--------+------+
only showing top 5 rows



574830

In [14]:
df_tickets_arrived = df_tickets_arrived.toDF(*map(str, range(len(df_tickets_arrived.columns))))

df_tickets_arrived.show(5)
df_tickets_arrived.count()

+-------------+-----+--------+--------+------+
|            0|    1|       2|       3|     4|
+-------------+-----+--------+--------+------+
|0005434877632|10096|Business|49700.00|PG0186|
|0005434878408|10096|Business|49700.00|PG0186|
|0005434878496|10096|Business|49700.00|PG0186|
|0005434878162|10096|Business|49700.00|PG0186|
|0005434877976|10096|Business|49700.00|PG0186|
+-------------+-----+--------+--------+------+
only showing top 5 rows



574830

In [15]:
df_tickets_arrived = df_tickets_arrived.withColumn('3', df_tickets_arrived['3'].cast(FloatType()))
df_tickets_arrived.show(5)

+-------------+-----+--------+-------+------+
|            0|    1|       2|      3|     4|
+-------------+-----+--------+-------+------+
|0005434877632|10096|Business|49700.0|PG0186|
|0005434878408|10096|Business|49700.0|PG0186|
|0005434878496|10096|Business|49700.0|PG0186|
|0005434878162|10096|Business|49700.0|PG0186|
|0005434877976|10096|Business|49700.0|PG0186|
+-------------+-----+--------+-------+------+
only showing top 5 rows



In [16]:
df_min_b = df_tickets_arrived.groupby('4', '2').min('3')
df_min_b = df_min_b[df_min_b['2'] == 'Business'].drop('2')

df_min_b.show(5)
df_min_b.count()

+------+--------+
|     4|  min(3)|
+------+--------+
|PG0274| 44500.0|
|PG0709|150400.0|
|PG0304| 26400.0|
|PG0305| 36600.0|
|PG0225| 43100.0|
+------+--------+
only showing top 5 rows



245

In [17]:
df_min_e = df_tickets_arrived.groupby('4', '2').min('3')
df_min_e = df_min_e[df_min_e['2'] == 'Economy'].drop('2')

df_min_e.show(5)
df_min_e.count()

+------+-------+
|     4| min(3)|
+------+-------+
|PG0224|14000.0|
|PG0486| 4200.0|
|PG0053|19300.0|
|PG0040| 3200.0|
|PG0278|28000.0|
+------+-------+
only showing top 5 rows



470

In [18]:
b_seats = df_tickets_arrived[df_tickets_arrived['2'] == 'Business'].groupby('4').count()
b_seats.show()

+------+-----+
|     4|count|
+------+-----+
|PG0144|  131|
|PG0325|    6|
|PG0278|  516|
|PG0503|  187|
|PG0088|   14|
|PG0211|  238|
|PG0383|   92|
|PG0690|  146|
|PG0412|  639|
|PG0539|   74|
|PG0230|  687|
|PG0521|  252|
|PG0108|   14|
|PG0311|  285|
|PG0588|  199|
|PG0217|  493|
|PG0397|  200|
|PG0115|  132|
|PG0571|  169|
|PG0186|  416|
+------+-----+
only showing top 20 rows



In [19]:
result = df_avg.join(df_min_b, df_avg['_c1'] == df_min_b['4']).drop(df_min_b['4'])
result.show(5)

+------+------------------+--------+
|   _c1|  avg(_c8-c2_unix)|  min(3)|
+------+------------------+--------+
|PG0144|193.84615384615384|203300.0|
|PG0325|             160.0| 91700.0|
|PG0088|             280.0| 92200.0|
|PG0278|1832.9032258064517| 84000.0|
|PG0503| 574.8387096774194| 67200.0|
+------+------------------+--------+
only showing top 5 rows



In [20]:
result = result.join(df_min_e, result['_c1'] == df_min_e['4']).drop(df_min_e['4'])
result.show(5)

+------+------------------+--------+-------+
|   _c1|  avg(_c8-c2_unix)|  min(3)| min(3)|
+------+------------------+--------+-------+
|PG0144|193.84615384615384|203300.0|67800.0|
|PG0325|             160.0| 91700.0|30600.0|
|PG0088|             280.0| 92200.0|30700.0|
|PG0278|1832.9032258064517| 84000.0|28000.0|
|PG0503| 574.8387096774194| 67200.0|22400.0|
+------+------------------+--------+-------+
only showing top 5 rows



In [21]:
result = result.join(b_seats, result['_c1'] == b_seats['4']).drop(b_seats['4'])
result.show(5)

+------+------------------+--------+-------+-----+
|   _c1|  avg(_c8-c2_unix)|  min(3)| min(3)|count|
+------+------------------+--------+-------+-----+
|PG0144|193.84615384615384|203300.0|67800.0|  131|
|PG0325|             160.0| 91700.0|30600.0|    6|
|PG0088|             280.0| 92200.0|30700.0|   14|
|PG0278|1832.9032258064517| 84000.0|28000.0|  516|
|PG0503| 574.8387096774194| 67200.0|22400.0|  187|
+------+------------------+--------+-------+-----+
only showing top 5 rows



In [22]:
result = result.toDF(*map(str, range(len(result.columns))))
result = result.na.fill(0)

result.show(5)
result.count()

+------+------------------+--------+-------+---+
|     0|                 1|       2|      3|  4|
+------+------------------+--------+-------+---+
|PG0144|193.84615384615384|203300.0|67800.0|131|
|PG0325|             160.0| 91700.0|30600.0|  6|
|PG0088|             280.0| 92200.0|30700.0| 14|
|PG0278|1832.9032258064517| 84000.0|28000.0|516|
|PG0503| 574.8387096774194| 67200.0|22400.0|187|
+------+------------------+--------+-------+---+
only showing top 5 rows



245

In [23]:
result = result.withColumn('score', 1 / result['1'] + 3 / 10000 * (result['2'] + result['3']) + 7 / 100 * result['4'])
result.orderBy(desc('score')).show(10)

+------+-----------------+--------+-------+---+------------------+
|     0|                1|       2|      3|  4|             score|
+------+-----------------+--------+-------+---+------------------+
|PG0208|            960.0|184500.0|61500.0|759|126.93104166666666|
|PG0209|            542.0|184500.0|61500.0|667|120.49184501845019|
|PG0356|            934.0|186200.0|62100.0|471|107.46107066381157|
|PG0357|843.8709677419355|186200.0|62100.0|415|103.54118501529052|
|PG0198|971.6129032258065|132900.0|44300.0|713|103.07102921646747|
|PG0703|            190.0|199300.0|66400.0|258| 97.77526315789473|
|PG0168|            178.0|199800.0|66600.0|224| 95.60561797752808|
|PG0704|            986.0|199300.0|66400.0|215| 94.76101419878296|
|PG0169|            182.0|199800.0|66600.0|170|  91.8254945054945|
|PG0199|            592.0|132900.0|44300.0|543| 91.17168918918918|
+------+-----------------+--------+-------+---+------------------+
only showing top 10 rows



In [24]:
result.orderBy(desc('score')).drop('1').drop('2').drop('3').drop('4').show(10)

+------+------------------+
|     0|             score|
+------+------------------+
|PG0208|126.93104166666666|
|PG0209|120.49184501845019|
|PG0356|107.46107066381157|
|PG0357|103.54118501529052|
|PG0198|103.07102921646747|
|PG0703| 97.77526315789473|
|PG0168| 95.60561797752808|
|PG0704| 94.76101419878296|
|PG0169|  91.8254945054945|
|PG0199| 91.17168918918918|
+------+------------------+
only showing top 10 rows



In [37]:
type(result)

pyspark.sql.dataframe.DataFrame